# Actor-Critic

## 0. Setups

In [1]:
colab = True

Import required libraries

For rendering **[COLAB USE ONLY!]**

In [2]:
if colab:
    !pip install swig
    !pip install gym pyvirtualdisplay > /dev/null 2>&1
    !apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
    !apt-get update > /dev/null 2>&1
    !apt-get install cmake > /dev/null 2>&1
    !pip install --upgrade setuptools 2>&1
    !pip install ez_setup > /dev/null 2>&1
    !pip3 install box2d-py
    !pip3 install gym[Box_2D]
    !pip install pygame

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 49.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 804.0/804.0 kB 49.7 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.5/374.5 kB 25.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for box2d-py: filename=box2d_py-2.3.8-cp310-cp310-linux_x86_64.whl size=2812299 sha256=3f82629c87e94d130717aa05958c545a8e22212fae49f4c3f670f642ee95db6d
  Stored in directory: /root/.cache/pip/wheels/47/01/d2/6a780da77ccb98b1d2facdd520a8d10838a03b590f6f8d50c0
Successfully built box2d-py
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import numpy as np
import gym
import time
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

from sklearn.linear_model import LinearRegression, Lasso

# For Colab users, turn this into true
colab = True

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


Build Environment and check MDP size

In [5]:
env = gym.make('CartPole-v1')
env.seed(500)
torch.manual_seed(500)

# Configure MDP
gamma = 0.99
state_dim = env.observation_space.low.size
num_action = env.action_space.n
print('Dimension of state space / number of actions : %d / %d'%(state_dim, num_action))

Dimension of state space / number of actions : 4 / 2


/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:256: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed)` instead.
  deprecation(


## 1. Create an policy instance

 Define policy network

In [6]:
class Policy(nn.Module):
    def __init__(self, state_dim, num_action, hidden_size1, hidden_size2):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(state_dim, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, num_action)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        action_score = self.fc3(x)
        return F.softmax(action_score, dim=1)

In [7]:
def select_action(state):
    state = torch.from_numpy(state).float().unsqueeze(0)
    state = state.to(device)
    probs = pi(state)
    m = Categorical(probs)
    action = m.sample()

    return action.item(), m.log_prob(action)

def sample_trajectory(data, T):
    # Reset environment to get new trajectory
    state = env.reset()
    r_sum, r_sum_discount = 0, 0

    for t in range(T):
        # Get action from current policy and rollout
        action, log_prob = select_action(state)
        next_state, reward, done, _ = env.step(action)
        r_sum += reward
        r_sum_discount += reward * (gamma ** t)

        # Store data
        data['log_pi'].append(-log_prob) # (-) sign for gradient ascent
        data['state'].append(state)
        data['next_state'].append(next_state)
        data['reward'].append(reward)

        # Step
        state = next_state
        if done:
            break

    return r_sum, r_sum_discount

## 3. Actor-Critic (Linear Architecture)

In [10]:
from numpy import linalg as LA

# Calculate feature vector
# state[0] : Cart pos
# state[1] : Cart speed
# state[2] : Pole angle
# state[3] : Pole velocity at tip

state2 = [-0.12, 0, 0.12] # termination condition
state3 = [-1, 0, 1]

mu = []
for s2 in state2:
    for s3 in state3:
        mu.append([s2, s3])

def state2feature(state):
    phi = []
    for f in mu:
        rad_base = LA.norm(np.array(state[-2:])-np.array(f)) ** 2
        phi.append(np.exp(-0.5*rad_base))
    return np.array(phi)


def calculate_vf(dataset, vf):
    X, y = [], []

    for data in dataset:
        for s, next_s, r in zip(data['state'], data['next_state'], data['reward']):
            v = state2feature(s)
            Q = r
            if vf is not None:
                # TO DO
                # Q =
                Q = r + gamma * vf.predict(state2feature(next_s).reshape(1, -1))[0]
            X.append(v)
            y.append(Q)

    return X, y


def get_advantage(data, vf):
    advantage, baseline = [], []

    for s, next_s, r in zip(data['state'], data['next_state'], data['reward']):
        # TODO: Complete advantage calculation by calculating Q-value
        # v =
        # v_next =
        v = vf.predict(state2feature(s).reshape(1, -1))[0]
        v_next = vf.predict(state2feature(next_s).reshape(1, -1))[0]

        Q = r + gamma * v_next
        A = Q - v

        advantage.append(A)
        baseline.append(v)

    return advantage, baseline


def calculate_AC_PG(vf, pi_returns_discounted, dataset):
    pi_loss = 0
    for data in dataset:
        # For linear Actor-Critic
        advantage = []
        _, v = get_advantage(data, vf)
        DCR = 0
        for i, r in enumerate(reversed(data['reward'])):
            DCR = r + gamma * DCR
            advantage.insert(0, DCR - v[i]) # For practical algorithm, we just adopt baseline

        # Compute each element of gradient
        pi_loss_linear_vf = [log_pi * a for log_pi, a in zip(data['log_pi'], advantage)]

        # Sums up log_prob * weight
        pi_loss += torch.cat(pi_loss_linear_vf).sum()

    return pi_loss / num_trajs

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
# num_epochs = 100
num_epochs = 15
num_trajs = 100
T = 10000
log_interval = 5
total_time = []

pi = Policy(state_dim, num_action, 128, 128).to(device)
optimizer_pi = optim.Adam(pi.parameters(), lr=1e-3)
vf = None

# For logging
pi_returns, pi_returns_discounted = [], []

dataset_vf = []
for epoch in range(num_epochs):
    start_epoch = time.time()

    # On-policy dataset
    dataset = []

    # Collect trajectories to perform gradient step
    for N in range(num_trajs):
        data = {'log_pi':[], 'state':[], 'next_state':[], 'reward':[]}
        r_sum, r_sum_discount = sample_trajectory(data, T)
        dataset.append(data)
        dataset_vf.append(data)

        # For logging - store most recent N trajectories
        pi_returns.append(r_sum)
        pi_returns_discounted.append(r_sum_discount)
        if len(pi_returns) > num_trajs:
            pi_returns.pop(0)
            pi_returns_discounted.pop(0)

    ### NEW : update critic ###
    X, y = calculate_vf(dataset_vf, vf)
    vf = LinearRegression().fit(X, y)

    # Perform pocliy gradient step
    optimizer_pi.zero_grad()
    pi_loss = calculate_AC_PG(vf, pi_returns_discounted, dataset)
    pi_loss.backward()
    optimizer_pi.step()

    # Logging - print most recent epoch result
    epoch_time = time.time() - start_epoch
    total_time.append(epoch_time)
    if epoch % log_interval == 0:
        dataset_vf = []
        time_elapsed = np.sum(total_time)
        time_remain = np.mean(total_time) * num_epochs - time_elapsed
        print('Epoch {}\tReturn_mean: {:.2f}\tReturn_std: {:.2f}\tTime(Elapsed/Remain): {:.2f}/{:.2f} (mins)'.format(
            epoch, np.mean(pi_returns), np.std(pi_returns), time_elapsed/60, time_remain/60))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Epoch 0	Return_mean: 23.50	Return_std: 11.22	Time(Elapsed/Remain): 0.13/1.85 (mins)
Epoch 5	Return_mean: 28.71	Return_std: 15.89	Time(Elapsed/Remain): 0.87/1.31 (mins)
Epoch 10	Return_mean: 38.94	Return_std: 20.65	Time(Elapsed/Remain): 1.82/0.66 (mins)


## 4. Visualize result

For rendering **[COLAB USE ONLY!]**

In [13]:
import gym
from gym.wrappers.record_video import RecordVideo
import glob
import io
import base64
from IPython.display import HTML
from pyvirtualdisplay import Display
from IPython import display as ipythondisplay

display = Display(visible=0, size=(1400, 900))
display.start()

"""
Utility functions to enable video recording of gym environment
and displaying it.
To enable video, just do "env = wrap_env(env)""
"""
def show_video(env_id):
    mp4list = glob.glob('video/'+env_id+'/*.mp4')
    if len(mp4list) > 0:
      mp4 = mp4list[0]
      video = io.open(mp4, 'r+b').read()
      encoded = base64.b64encode(video)
      ipythondisplay.display(HTML(data='''<video alt="test" autoplay
                  loop controls style="height: 400px;">
                  <source src="data:video/mp4;base64,{0}" type="video/mp4" />
              </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")


def wrap_env(env, env_id):
#    env = Monitor(env, './video', force=True)
    env=RecordVideo(env, './video/'+env_id,  episode_trigger = lambda episode_number: True)
    return env

In [14]:
env_id = 'CartPole-v1'
env = gym.make(env_id)
if colab:
  env = wrap_env(env, env_id)
state = env.reset()

while True:
    env.render()
    action, log_prob = select_action(state)
    state, reward, done, info = env.step(action)
    if done:
        break

env.close()
if colab:
    show_video(env_id)

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/monitoring/video_recorder.py:78: DeprecationWarning: WARN: Recording ability for environment CartPole-v1 initialized with `render_mode=None` is marked as deprecated and will be removed in the future.
  logger.deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is dep